# Error analysis and (better) testing

We train a text classification model on financial news (mostly as a black-box, focus is not on modelling), and then use the model to understand some best practices in error analysis and testing.

You are encouraged to play around with the code and modify / re-built parts of it as you fit: there is NO substitute for "tinkering with code" to understand how all the concepts fit together (corollary: all this code is written for pedagogical purposes, so some functions are re-used from previous lectures to provide a self-sufficient script).

_First, let's make sure we are running from the virtual env_

In [ ]:
import sys
sys.executable

In [ ]:
# some global import
# we import specific libraries in due time
import json
import glob
import os
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from random import choice

In [ ]:
%matplotlib inline

## Data loading

In [ ]:
# make sure you have the datasets library installed
# see: https://github.com/huggingface/datasets

# !pip install datasets

In [ ]:
import string

# some utils function
def get_finance_sentiment_dataset(split: str='sentences_allagree'):
    # load financial dataset from HF
    from datasets import load_dataset
    # https://huggingface.co/datasets/financial_phrasebank
    # by default, load just sentences for which all annotators agree
    dataset = load_dataset("financial_phrasebank", split)
    
    return dataset['train']


def get_finance_sentences():
    dataset = get_finance_sentiment_dataset()
    cleaned_dataset = [[pre_process_sentence(_['sentence']), _['label']] for _ in dataset]
    # debug 
    print("{} cleaned sentences from finance dataset\n".format(len(cleaned_dataset)))
    
    return cleaned_dataset


def pre_process_sentence(sentence: str):
    # this choices are VERY important. Here, we take a simplified 
    # view, remove the punctuations and just lower case everything
    lower_sentence = sentence.lower()
    # remove punctuation
    # nice suggestion from https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    # if we change the exclude set, we can control what to exclude
    exclude = set(string.punctuation)
    return ''.join(ch for ch in lower_sentence if ch not in exclude)

In [ ]:
finance_dataset = get_finance_sentences()
# print out the first item in the dataset, to check the format
finance_dataset[0]

## Model training

Training a simple tf-idf classifier on text and return the model for analysis, prediction. More info about the model:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Some more details on NLP classification can be found for example in our 2021 course: https://github.com/jacopotagliabue/FREE_7773


In [ ]:
# first get the final dataset splits etc.
from sklearn.model_selection import train_test_split

finance_dataset_text = [_[0] for _ in finance_dataset]
finance_dataset_label = [_[1] for _ in finance_dataset]
all_labels = set(finance_dataset_label)
print("All labels are: {}".format(all_labels))
X_train, X_test, y_train, y_test = train_test_split(finance_dataset_text, 
                                                    finance_dataset_label, 
                                                    test_size=0.1, 
                                                    random_state=42)
print("Total train examples {}, test {}".format(len(X_train), len(y_train)))
# debug with examples
print(X_train[:2], y_train[:2])

*NOTE: LABELS ARE 0 NEG, 1 NEUTRAL AND 2 POSITIVE !*

In [ ]:
def train_model(tf_idf_x_train, y_train):
    """
    Train a simple classifier over the text vectors.
    Model from https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html
    """
    from sklearn.naive_bayes import MultinomialNB
    model = MultinomialNB()
    model.fit(tf_idf_x_train, y_train)
    
    return model

def get_trained_classifier(X_train, y_train):
    """
    Encapsulate the training here, as we really don't care about training details:
    the model is just useful to discuss testing strategies!
    """
    
    # map text to numerical vectors using TF-IDF and some sensible defaults
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
    tfidf_train = vectorizer.fit_transform(X_train)
    # debug: what does this shape mean?
    print(tfidf_train.shape)
    # train the model
    model = train_model(tfidf_train, y_train)
        
    # since we are treating of all this 
    return vectorizer, model

In [ ]:
tf_vectorizer, clf_model = get_trained_classifier(X_train, y_train)

## Evaluate the model

How well are we doing?

First, we now instantiate a classifier, train it and then predicting unseen test cases as usual.

In [ ]:
X_test_transformed = tf_vectorizer.transform(X_test)
predicted = clf_model.predict(X_test_transformed)
predicted_prob = clf_model.predict_proba(X_test_transformed)
# debug output
print(predicted[:2], predicted_prob[:2])

We start by using standard quantitative metrics to evaluate our model

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def calculate_confusion_matrix_and_report(y_predicted, y_golden, with_plot=True):
    # calculate confusion matrix: 
    cm = confusion_matrix(y_golden, y_predicted)
    # build a readable report;
    # https://scikit-learn.org/stable/modules/model_evaluation.html#classification-report
    print('\nClassification Report')
    print(classification_report(y_golden, y_predicted))
    # plot the matrix
    if with_plot:
        plot_confusion_matrix(cm)
                                          
    return
                                          
def plot_confusion_matrix(c_matrix):
    plt.imshow(c_matrix, cmap=plt.cm.Blues)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.xticks([], [])
    plt.yticks([], [])
    plt.title("Confusion matrix")
    plt.colorbar()
    plt.show()
    
    return

In [ ]:
print("Total of # {} test cases".format(len(y_test)))
calculate_confusion_matrix_and_report(predicted, y_test)

### Error analysis

In [ ]:
assert len(X_test) == len(predicted)
# manual inspection
mistakes = [(x, p, y, prob) for x, p, y, prob in zip(X_test, predicted, y_test, predicted_prob) if p != y]
print("Total of mistakes: {}".format(len(mistakes)))
# debug
print("Sentence: {}\nPredicted: {}, but it was: {}\nProbs: {}".format(*mistakes[0]))

In [ ]:
for _ in range(3):
    rnd_mistake = choice(mistakes)
    print("Sentence: {}\nPredicted: {}, but it was: {}\nProbs: {}\n=======\n".format(*rnd_mistake))

Now, let's run evaluation *per slice*: instead of considering the performances on all dataset, we split it according to categories important for our use case.

In our example, we will assume we are interested in being accurate across all quarters, so we now report our metrics per slice / quarter.

You can imagine many more relevant slices: report by industry (pharma, tech, etc.), market cap (over a 1B, over 100 etc.).

In [ ]:
# let's say we slice queries by quarter
slices = {
    "first quarter": [[], []],
    "second quarter": [[], []],
    "third quarter": [[], []],
    "fourth quarter": [[], []]
}

for x, p, y in zip(X_test, predicted, y_test):
    for _s in slices.keys():
        if _s in x:
            slices[_s][0].append(p)
            slices[_s][1].append(y)
            
for _slice, test_set in slices.items():
    if test_set[0]:
        print("Total of # {} cases in slice: {}".format(len(test_set[0]), _slice))
        calculate_confusion_matrix_and_report(test_set[0], test_set[1], with_plot=False)
        print("\n===========\n")

## Black-box testing

Sometime, we wish to test our models on edge cases not present in the test set, or on specific conditions we know are important to safely deploy it: for example, if we work for company X, we may want to "double check" the behavior of the system in carefully crafted stories about company X, to make sure the model WOULD behave correctly, if presented with those cases.

More specifically, we will adapt the “black box testing” from traditional software systems to ML systems: it should be possible to evaluate the performance of a complex system by treating it as a black box, and only supply input-output pairs that are relevant for our qualitative understanding (see for example the excellent paper: https://arxiv.org/abs/2005.04118).

In [ ]:
# Test for edge cases / interesting cases / regression errors

# CASE 1:
# I'm particularly interesting in some company, say
# https://en.wikipedia.org/wiki/Comptel
# and want to make sure we are doing well there!

companies_I_care_about = ['comptel']

for company in companies_I_care_about:
    print("\n======\nFocus on target company: {}\n".format(company))
    for x, p, y in zip(X_test, predicted, y_test):
        if company in x:
            print("For '{}' =>\ngolden {}, predicted {}\n".format(
                x,
                y, 
                p))

In [ ]:
# CASE 2:
# Assuming we have some specific sentences to monitor, let's check for that!
sentences_I_care_about = [
    'the company slipped to an operating loss of eur 26 million from a profit of eur 13 million',
    'revenue in the quarter fell 8 percent to  euro  24 billion compared to a year earlier'
]
labels_I_care_about = [0, 0]

for x, p in zip(X_test, predicted):
     if x.strip() in sentences_I_care_about:
        print("For '{}', I expect {}, it was {}\n".format(
            x,
            p, 
            labels_I_care_about[sentences_I_care_about.index(x.strip())]
        ))

*BONUS: test for robustness*

Ideally, a model should perform the same when input change in a small way (check this paper on "adversarial attacks": https://arxiv.org/pdf/1412.6572.pdf!).

For example, in test classification, we desire to have a system robust to alternative specifications of the text, i.e. we expect the response to the pair ("revenue in the quarter fell 8 percent to  euro  24 billion compared to a year earlier", "revenue in the quarter diminished by 8 percent to  euro  24 billion compared to the previous year") to be identical.

While a full-fledge treatment of this problem is out of scope, let's see how this intuition works with working code.

In [ ]:
# test for perturbations
test_sentences = [
    'the company slipped to an operating loss of eur 26 million from a profit of eur 13 million',
    'revenue in the quarter fell 8 percent to  euro  24 billion compared to a year earlier'
]

perturbated_sentences = [
    'operating loss surged to eur 26 million from a profit of eur 13 million',
    'revenue in the quarter diminished by 8 percent to  euro  24 billion compared to the previous year'
]

test_predicted = clf_model.predict(tf_vectorizer.transform(test_sentences))
perturbated_predicted = clf_model.predict(tf_vectorizer.transform(perturbated_sentences))

for s, t, p in zip(test_sentences, test_predicted, perturbated_predicted):
    print("\nFor sentence '{}', prediction was: {}, under perturbation: {}".format(
        s, t, p
    ))

How do we scale this to more and more examples? A quick and easy way to generate perturbation is called back-translation.

The idea is that you can use machine translation to go:

SOURCE -> TARGET -> NEW_SOURCE

where NEW_SOURCE is a semantically equivalent, but not identical version of source. For example:

'hi' -> Italian target: 'ciao' -> 'hello'

'hi' and 'hello' have the same meaning so, 'hello' may be considered a perturbation of the original text.

In [ ]:
# !pip install BackTranslation

In [ ]:
# this is really small hack-y example!!!
from BackTranslation import BackTranslation
trans = BackTranslation(url=[
      'translate.google.com',
      'translate.google.co.kr',
    ])
for t in test_sentences:
    result = trans.translate(t, src='en', tmp = 'zh-cn')
    print("Original is: {}\nNew sentence is: {}\n\n".format(t, result.result_text))